# ARC AGI Visualizer and Concept Tagger
This notebook allows you to visualize ARC AGI tasks and tag them with required concepts.
- Use the `Previous` and `Next` buttons to navigate.
- Type concept tags and save them to a JSON file.

In [1]:
import os
import json
import requests
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets
from IPython.display import display, clear_output

In [19]:
import os
import requests
from bs4 import BeautifulSoup

def get_arc_task_ids(split="training"):
    url = f"https://github.com/fchollet/ARC/tree/master/data/{split}"
    raw_prefix = f"https://raw.githubusercontent.com/fchollet/ARC/master/data/{split}/"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    task_ids = [a.text.replace(".json", "") for a in soup.find_all("a") if a.text.endswith(".json")]
    return task_ids

training_ids = get_arc_task_ids("training")
print(f"Loaded {len(training_ids)} task IDs.")
print(training_ids[:5])  # show a sample


Loaded 0 task IDs.
[]


In [20]:
import os
import subprocess

# 1. Clone repository (only once)
if not os.path.isdir("ARC-AGI"):
    subprocess.run(["git", "clone", "--depth", "1",
        "https://github.com/fchollet/ARC-AGI.git"])

# 2. List training task IDs
task_dir = "ARC-AGI/data/training"
task_ids = [fname[:-5] for fname in os.listdir(task_dir) if fname.endswith(".json")]
print(f"Found {len(task_ids)} tasks, e.g.: {task_ids[:5]}")


Found 400 tasks, e.g.: ['007bbfb7', '00d62c1b', '017c7c7b', '025d127b', '045e512c']


In [ ]:
# Sample ARC tasks
ARC_AGI_URL = "https://raw.githubusercontent.com/fchollet/ARC/master/data/training/"
TASK_IDS = [
    '0b148d64', '0c786b71', '0d3d703e', '0dfd9992', '0e206a2e'
]
os.makedirs("arc_tasks", exist_ok=True)
#for task_id in TASK_IDS:
for task_id in task_ids:
    url = f"{ARC_AGI_URL}{task_id}.json"
    r = requests.get(url)
    with open(f"arc_tasks/{task_id}.json", 'w') as f:
        f.write(r.text)

In [ ]:

import matplotlib.colors as mcolors

# Fixed color map for ARC values 0–9


def plot_grid(grid, ax):
    #cmap = mcolors.ListedColormap([
    #    "#000000", "#0074D9", "#2ECC40", "#FFDC00", "#FF4136",
    #    "#AAAAAA", "#F012BE", "#FF851B", "#7FDBFF", "#870C25"
    #])
    #cmap = mcolors.ListedColormap([
    #"#000000",  # 0 - black
    #"#1f77b4",  # 1 - blue
    #"#ff7f0e",  # 2 - orange
    #"#2ca02c",  # 3 - green
    #"#d62728",  # 4 - red
    #"#9467bd",  # 5 - purple
    #"#8c564b",  # 6 - brown
    #"#e377c2",  # 7 - pink
    #"#17becf",  # 8 - cyan
    #"#bcbd22"   # 9 - olive
    #])

    cmap = mcolors.ListedColormap([
    "#000000",  # 0 - black
    "#1f77b4",  # 1 - blue
    "#ff7f0e",  # 2 - orange
    "#2ca02c",  # 3 - green
    "#d62728",  # 4 - red
    "#9467bd",  # 5 - purple
    "#8c564b",  # 6 - brown
    "#e377c2",  # 7 - pink
    "#17becf",  # 8 - cyan
    "#bcbd22"   # 9 - olive/lime
    ])
    norm = mcolors.BoundaryNorm(boundaries=range(11), ncolors=10)
    ax.imshow(grid, cmap=cmap, norm=norm, interpolation='none')
    #ax.imshow(grid, cmap=cmap, interpolation='none')
    ax.set_xticks([])
    ax.set_yticks([])
    print(grid)

In [ ]:
class ARCViewer:
    def __init__(self, task_dir="arc_tasks", tag_file="concept_tags.json"):
        self.task_files = sorted(os.listdir(task_dir))
        self.index = 0
        self.task_dir = task_dir
        self.tag_file = tag_file
        self.tags = self.load_tags()
        self.tag_input = widgets.Text(description='Tags:')
        self.save_btn = widgets.Button(description='Save Tags')
        self.output = widgets.Output()
        self.save_btn.on_click(self.save_tags)
        self.update_display()
        self.create_widgets()

    def load_tags(self):
        if os.path.exists(self.tag_file):
            with open(self.tag_file) as f:
                return json.load(f)
        return {}

    def save_tags(self, _):
        task_id = self.task_files[self.index].replace('.json', '')
        self.tags[task_id] = [t.strip() for t in self.tag_input.value.split(',') if t.strip()]
        with open(self.tag_file, 'w') as f:
            json.dump(self.tags, f, indent=2)
        with self.output:
            print(f"Saved tags for {task_id}: {self.tags[task_id]}")

    def create_widgets(self):
        prev_btn = widgets.Button(description="Previous")
        next_btn = widgets.Button(description="Next")
        prev_btn.on_click(self.prev_task)
        next_btn.on_click(self.next_task)
        display(widgets.HBox([prev_btn, next_btn, self.tag_input, self.save_btn]), self.output)

    def load_task(self, index):
        with open(os.path.join(self.task_dir, self.task_files[index])) as f:
            return json.load(f)

    def update_display(self):
        with self.output:
            clear_output(wait=True)
            task_id = self.task_files[self.index].replace('.json', '')
            task = self.load_task(self.index)
            fig, axes = plt.subplots(len(task["train"]), 2, figsize=(5, 2 * len(task["train"])))
            fig.suptitle(f"Task: {task_id}", fontsize=14)
            for i, pair in enumerate(task["train"]):
                print(i,"input")
                plot_grid(pair["input"], axes[i, 0])
                print(i,"output")
                plot_grid(pair["output"], axes[i, 1])
                axes[i, 0].set_title("Input")
                axes[i, 1].set_title("Output")
            self.tag_input.value = ', '.join(self.tags.get(task_id, []))
            plt.tight_layout()
            suggestions = suggest_concepts(task["train"][0])
            print(f"Suggested concepts: {', '.join(suggestions)}")            
            plt.show()

    def prev_task(self, _):
        if self.index > 0:
            self.index -= 1
            self.update_display()

    def next_task(self, _):
        if self.index < len(self.task_files) - 1:
            self.index += 1
            self.update_display()

viewer = ARCViewer()

In [ ]:
def suggest_concepts(pair):
    input_grid = pair['input']
    output_grid = pair['output']

    concepts = set()
    if len(input_grid) != len(output_grid) or len(input_grid[0]) != len(output_grid[0]):
        concepts.add("size change")
    
    unique_colors_in = set([cell for row in input_grid for cell in row])
    unique_colors_out = set([cell for row in output_grid for cell in row])
    if unique_colors_in != unique_colors_out:
        concepts.add("color transformation")

    if input_grid == output_grid:
        concepts.add("identity")
    
    if any(row != output_grid[i] for i, row in enumerate(input_grid)):
        concepts.add("pattern shift or symmetry")
    
    return list(concepts)